In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import convert_all as crd

In [10]:
# get all low_dim data (head&spine velocity control)
task_name = "mmk2_test"
raw_root_dir = "../data/raw"
raw_dir = f"{raw_root_dir}/{task_name}"
data = crd.raw_to_dict(
    raw_dir,
    ["low_dim.json"],
    video_file_names=None,
    flatten_mode="hdf5",
    concatenater={
        "/observations/qpos": (
            "/observation/arm/left/joint_position",
            "/observation/eef/left/joint_position",
            "/observation/arm/right/joint_position",
            "/observation/eef/right/joint_position",

        ),
        "/action": (
            "/action/arm/left/joint_position",
            "/action/eef/left/joint_position",
            "/action/arm/right/joint_position",
            "/action/eef/right/joint_position",
        ),
    },
    key_filter=[
        "/observation/ts_diff_with_head_color_img",
        "/observation/arm/left/joint_velocity",
        "/observation/arm/right/joint_velocity",
        "/observation/arm/left/joint_effort",
        "/observation/arm/right/joint_effort",
        "/observation/eef/left/joint_velocity",
        "/observation/eef/right/joint_velocity",
        "/observation/eef/left/joint_effort",
        "/observation/eef/right/joint_effort",
        "/observation/head/joint_position",
        "/observation/head/joint_velocity",
        "/observation/head/joint_effort",
        "/observation/spine/joint_position",
        "/observation/spine/joint_velocity",
        "/observation/joint_states/time",
        "/observation/time",
        "/action/time",
        "/action/arm/left/time",
        "/action/arm/right/time",
        "/action/head/color/time",
        "/action/head/joint_position",
        "/action/spine/joint_position",
        # "/action/head/joint_velocity",
        # "/action/spine/joint_velocity"
    ],
)

Data Converting: 100%|██████████| 8/8 [00:00<00:00, 63.04it/s]


In [11]:
ep_number = len(data)
print(f"Number of episodes: {ep_number}")

Number of episodes: 8


In [12]:
first_ep = list(data.values())[0]
print(list(first_ep.keys()))
print(list(first_ep["/observations/qpos"][0]))
print(list(first_ep["/action"][0]))

['/observations/qpos', '/action']
[-0.04291599988937378, -0.7013428211212158, 1.2224383354187012, 2.0181963443756104, -1.1190584897994995, -0.2615014910697937, 0.029921988025307655, 0.17948424816131592, -0.8020523190498352, 1.3544288873672485, -1.9338903427124023, 1.185053825378418, 0.34924086928367615, 0.04041256383061409]
[-0.049100000000000144, -0.7028670748005672, 1.2214335374002836, 1.9853999999999994, -1.1118000000000001, -0.26670000000000016, -0.026499999999999968, 0.17989999999999995, -0.8085670748005673, 1.3583335374002834, -1.8946000000000003, 1.1762999999999995, 0.3562999999999996, 0.0025999999999997137]


In [ ]:
import os
import cv2

# merge high_dim data and save
raw_dir
names = ["head"]
video_names = [f"{name}.mp4" for name in names]
target_root_dir = "../data/hdf5"
target_dir = f"{target_root_dir}/{task_name}"
low_dim_data = data
name_converter = {names[i]: f"/observations/images/{i}" for i in range(len(names))}
target_namer = lambda i: f"episode_{i}.hdf5"

compresser = crd.Compresser("jpg", [int(cv2.IMWRITE_JPEG_QUALITY), 50], True)

os.makedirs(target_dir, exist_ok=True)

# get max episode length
episode_lens = []
for key, low_d in low_dim_data.items():
    length = len(list(low_d.values())[0])
    episode_lens.append(length)
    # if length < 200:
    #     print(f"{key} has length {length}")

max_pad_length = max(episode_lens)

# save all data
episode_names = list(low_dim_data.keys())
print(f"Episode lengths: {episode_lens}")
print(f"Max episode length: {max_pad_length}")
print(f"All episodes: {episode_names}")
print(f"episode number: {len(episode_names)}")
downsampling = 0
def save_one(index, ep_name):
    crd.merge_video_and_save(
        low_dim_data[ep_name],
        f"{raw_dir}/{ep_name}",
        video_names,
        crd.save_dict_to_hdf5,
        name_converter,
        compresser,
        f"{target_dir}/" + target_namer(index),
        max_pad_length,
        downsampling
    )
    data.pop(ep_name)

In [ ]:
# save all
from concurrent.futures import ThreadPoolExecutor
futures = []
with ThreadPoolExecutor(max_workers=25) as executor:
    for index, ep_name in enumerate(episode_names):
        # silent execution, no print
        futures.append(executor.submit(save_one, index, ep_name))
print(f"All data saved to {target_dir}")

In [ ]:
save_one(0, episode_names[0])

In [ ]:
# check hdf5 data
import convert_all as crd
# task_name = "1block1bowl"
# target_root_dir = "../data/hdf5"
target_dir = f"{target_root_dir}/{task_name}"
data = crd.hdf5_to_dict(f"{target_dir}/episode_0.hdf5")
print(data.keys())
data_flat = crd.flatten_dict(data, prefix="/")
print(data_flat.keys())

In [ ]:
# print(data_flat['/compressed_len'])
print(len(data_flat["/observations/images/0"]))
print(len(data_flat["/action"]))

In [ ]:
import cv2
image = data_flat["/observations/images/0"][0]
print(image.shape)
compresser = crd.Compresser("jpg", [int(cv2.IMWRITE_JPEG_QUALITY), 50], True)
image_dc = compresser.decompress(image, "jpg")
print(image_dc.shape)
import matplotlib.pyplot as plt
plt.imshow(image_dc[:, :, ::-1])
plt.show()

In [ ]:
# fix action
import json
import numpy as np

for i in range(49, 50):
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "r") as f:
        data = json.load(f)
        action = data["action"]["spine"]["joint_position"]
        obs = data["observation"]["spine"]["joint_position"]
        # change action
        delta = np.array(obs[1:]) - np.array(obs[:-1])
        # > 0 -> 1, < 0 -> -1
        action = np.sign(delta)
        # 获取不为0的值的index
        index_not0 = np.where(action != 0)
        first_not0 = index_not0[0][0]
        last_not0 = index_not0[0][-1]
        # 从第一个不为0的值开始，到最后一个不为0的值结束，将中间的值设为1
        action[first_not0:last_not0] = 1
        action = action.tolist()
        action.append(action[-1])
        data["action"]["spine"] = {"joint_velocity": action}
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "w") as f:
        json.dump(data, f)

In [ ]:
# rename
import json

for i in range(50):
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "r") as f:
        data = json.load(f)
        data["action"]["head"] = {"joint_velocity": data["action"]["head"]["joint_velocity"]["joint_position"]}
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "w") as f:
        json.dump(data, f)

## Position Control

In [ ]:
# add head and spine to joint_position action
import json

for i in range(58):
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "r") as f:
        data = json.load(f)
        obs_spine = data["observation"]["spine"]["joint_position"]
        obs_head = data["observation"]["head"]["joint_position"]
        data["action"]["head"]["joint_position"] = obs_head[1:] + [obs_head[-1]]
        data["action"]["spine"]["joint_position"] = obs_spine[1:] + [obs_spine[-1]]
    with open(f"demonstrations/raw/mmk_pick_grape/{i}/low_dim.json", "w") as f:
        json.dump(data, f)

In [ ]:
# get all low_dim data (head&spine position control)
import convert_all as crd
raw_dir = "demonstrations/raw/mmk_pick_grape"
data = crd.raw_to_dict(
    raw_dir,
    ["low_dim.json"],
    video_file_names=None,
    flatten_mode="hdf5",
    concatenater={
        "/observations/qpos": (
            "/observation/arm/left/joint_position",
            "/observation/eef/left/joint_position",
            "/observation/arm/right/joint_position",
            "/observation/eef/right/joint_position",
            "/observation/head/joint_position",
            "/observation/spine/joint_position"
        ),
        "/action": (
            "/action/arm/left/joint_position",
            "/action/eef/left/joint_position",
            "/action/arm/right/joint_position",
            "/action/eef/right/joint_position",
            "/action/head/joint_position",
            "/action/spine/joint_position"
        ),
    },
    key_filter=[
        "/observation/arm/left/joint_velocity",
        "/observation/arm/right/joint_velocity",
        "/observation/arm/left/joint_torque",
        "/observation/arm/right/joint_torque",
        "/action/head/joint_velocity",
        "/action/spine/joint_velocity"
    ],
)

In [ ]:
ep_number = len(data)
print(f"Number of episodes: {ep_number}")

In [ ]:
# 将spine的observation和action[-120000, 0]缩放到[-2pi, 0]
import numpy as np
for i in range(ep_number):
    qpos_obs = np.array(data[str(i)]["/observations/qpos"])
    qpos_act = np.array(data[str(i)]["/action"])
    qpos_obs[:, -1] = 2 * np.pi * qpos_obs[:, -1] / 120000
    qpos_act[:, -1] = 2 * np.pi * qpos_act[:, -1] / 120000
    data[str(i)]["/observations/qpos"] = qpos_obs.tolist()
    data[str(i)]["/action"] = qpos_act.tolist()